In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook was authored with assistance from [Ivan Nardini](https://github.com/inardini)

# E2E ML on GCP: MLOps stage 2 : experimentation: get started with Vertex AI Experiments

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_experiments.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 2 : experimentation: get started with Vertex AI Experiments.

### Objective

In this tutorial, you learn how to use `Vertex AI Experiments` when training with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Experiments`
- `Vertex ML Metadata`
- `Vertex AI Training`

The steps performed include:

- Local (notebook) Training
    - Create an experiment
    - Create a first run in the experiment
    - Log parameters and metrics
    - Create artifact lineage
    - Visualize the experiment results
    - Execute a second run
    - Compare the two runs in the experiment
- Cloud (`Vertex AI`) Training
    - Within the training script:
        - Create an experiment
        - Log parameters and metrics
        - Create artifact lineage
    - Create a `Vertex AI Training` custom job
    - Execute the custom job
    - Visualize the experiment results

### Recommendations

When doing E2E MLOps on Google Cloud, the following are some of the best practices for logging data when experimenting or formally training a model.

#### Python Logging

Use Python's logging package when doing ad-hoc training locally.

#### Cloud Logging

Use `Google Cloud Logging` when doing training on the cloud.

#### Experiments

Use Vertex AI Experiments in conjunction with logging when performing experiments to compare results for different experiment configurations.

### Dataset

This tutorial does not use a dataset. References to example datasets is for demonstration purposes.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Installations

Install the following packages for executing this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install {USER_FLAG} --upgrade gsutil -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI, Compute Engine, Cloud Storage and Cloud Logging APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component,logging).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

1. **Click Create service account**.

2. In the **Service account name** field, enter a name, and click **Create**.

3. In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex AI" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

4. Click Create. A JSON file that contains your key downloads to your local environment.

5. Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries

In [ ]:
import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training.

Set the variables `TRAIN_GPU/TRAIN_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more about [hardware accelerator support for your region](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

#### Set pre-built containers

Set the pre-built Docker container image for training.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)


TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### Set machine type

Next, set the machine type to use for training.

- Set the variable `TRAIN_COMPUTE` to configure  the compute resources for the VMs you will use for for training.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## Introduction to `Vertex AI Experiments`

With `Vertex AI Experiments` you can log and track the following when experimenting/devloping your model architecture and model training:

- Log the metaparameters for the model architecture.
- Log the hyperparameters for training.
- Log the evaluation metrics.
- Create an artifact lineage of the dataset, model and evaluation.
- Group one or more training runs under an experiment.
- Compare experiments.

`Vertex AI Experiments` can be integrated with the following development process flows:

- Local development in a notebook
- Cloud development in `Vertex AI Training`
- Operationalizing development in `Vertex AI Pipelines`

Learn more about [Experiments](https://cloud.google.com/vertex-ai/docs/experiments/).

Learn more about [Introduction to Vertex AI ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

### Local development in a notebook

You can track an experiment in your local development, such as in a Vertex AI Workbench notebook, by:

- Wrap (preamble) the creation of an experiment.
- Instantiate a run per training run in the experiment.
- Within the local training run, log the corresponding parameters and results.
- Create lineage to the artifacts and experiment data.
- Retreive the experiment data.

#### Create experiment for tracking training related metadata

First, you create an experiment using the `init()` method and then initialize a run within the experiment using `start_run()`.

- `aiplatform.init()` - Create an experiment instance
- `aiplatform.start_run()` - Track a specific run within the experiment.

In [ ]:
# Specify a name for the experiment
EXPERIMENT_NAME = "[your-experiment-name]"

if EXPERIMENT_NAME == "[your-experiment-name]":
    EXPERIMENT_NAME = "example-" + TIMESTAMP

In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

#### Log parameters for the experiment

Typically, an experiment is associated with a specific dataset and a model architecture. Within an experiment, you may have multiple training runs, where each run tries a different configuration. For example:

- Data feeding, such as:
    - Dataset split
    - Dataset sampling and boosting
- Metaparameters, such as:
    - Depth and width of layers
- Hyperparameters, such as:
    - batch size
    - learning rate

These configuration settings are referred to as parameters, which you store as key-value pairs using the method `log_params()`

In [ ]:
metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

#### Log metrics for the experiment

At the completion or termination of a run within an experiment, you can log results that you use to compare runs. For example:

- Evaluation metrics
- Hyperparameter search selection
- Time to train the model
- Early stop trigger

These results are referred to as metrics, which you store as key-value pairs using the method `log_metrics()`

In [ ]:
metrics = {}
metrics["test_acc"] = 98.7
metrics["train_acc"] = 99.3
aiplatform.log_metrics(metrics)

#### Get the experiment results

When you are finished with a run within an experiment, you call `end_run()` method to complete the logging for that run.

Next, you use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

#### Start subsequent run in an experiment

Next, you create a second run for the same experiment. In this example, you change the metaparameter for `units` from 126 to 256, and log different metric results.

In [ ]:
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = 256  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = 98.8  # value changed
metrics["train_acc"] = 99.5  # value changed
aiplatform.log_metrics(metrics)

#### Comparing runs in the same experiment

Finally, you use the experiment name as a parameter to the method `get_experiment_df()` to get the results of all the runs within the experiment as a pandas dataframe.

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

#### Delete the experiment

Next, you delete the experiment using the `delete()` method.

In [ ]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
exp.delete()

### Create artifact lineage in experiment runs

In this example, you add artifact lineage to your experiment run. First, you create an experiment and then start a run within the experiment.

In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

#### Create a dataset and model artifacts

Next, you create synthetic artifacts in the Vertex AI ML Metadata to associated with this run in the experiment, as lineage. You will create:

- `dataset_artifact`: A dataset that is the input to the experiment run.
- `model_artifact`: A model that is the output from the experiment run.

In [ ]:
DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

#### Create the artifact lineage

Next, to create artifact lineage for an experiment run, you instantiate an execution using the method `start_execution()`. You then attach input artifacts using the method `assign_input_artifacts()` and attach output artifacts using the method `assign_output_artifacts()`.

In this example, to find the lineage for the experiment, you add a synthetic (metadata) entry `lineage` to the execution run, and set the value to the console uri for the lineage, which you can get from the method `get_output_artifacts()` and the property `lineage_console_uri`.

In [ ]:
with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    aiplatform.log_params({"units": 256})
    aiplatform.log_metrics({"acc": 96.8})

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

#### Get the experiment results

Next, you use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In this example, you stored the resource URI to the lineage as a metric value `lineage` in the execution run.

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

#### Visualize the artifact lineage

Next, open the link below to visualize the artifact lineage.

In [ ]:
print(
    "Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri
)

#### Delete the artifact lineage

Next, use the delete() method to delete the artifact lineage.

In [ ]:
try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

#### Delete the experiment

Next, you delete the experiment using the `delete()` method.

In [ ]:
exp.delete()

### Cloud development in `Vertex AI Training`

You can track an experiment in your cloud development using `Vertex AI Training`, by:

In your Python training script, repeat the same steps as in local development:

- Wrap (preamble) the creation of an experiment.
- Instantiate a run per training run in the experiment.
- Within the local training run, log the corresponding parameters and results.
- Create lineage to the artifacts and experiment data.
- Retreive the experiment data.

#### Package layout

Before you start the training, you will look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the Docker image.

The file `trainer/task.py` is the Python script for executing the custom training job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Package Assembly

In the following cells, you will assemble the training package.

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'google-cloud-aiplatform',\n\n  ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Synethic Training Script for Experiments\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Create synthetic training script

First, you write a synthetic training script. It won't actually train a model, but instead mimics the training of the model:

- Argument parsing
  - `experiment`: The name of the experiment.
  - `run`: The name of the run within the experiment.
  - `epochs`: The number of epochs.
  - `dataset-uri`: The Cloud Storage location of the training data.
  - `model-dir`: The Cloud Storage location to save the trained model artifacts.
- Training functions
  - `get_data()`: 
      - Get the training data. 
      - Create the input dataset artifact.
      - Attach dataset artifact as input to execution context.
  - `get_model()`:
      - Get the model architecture.
  - `train_model()`:
      - Train the model
  - `save_model()`:
      - Save the model
      - Create the output model artifact.
      - Attach model artifact as output to execution context.
- Initialize the experiment (`init()`) and start a run (`start_run()`) within the experiment
- Wrap the training with a `start_execution()`.
- Log the lineage to the experiment parameters (`log_metrics({"lineage"...)`)
- End the experiment run (`end_run()`)

In [ ]:
%%writefile custom/trainer/task.py

import argparse
import os

import google.cloud.aiplatform as aiplatform

parser = argparse.ArgumentParser()
# Args for experiment
parser.add_argument('--experiment', dest='experiment',
                    required=True, type=str,
                    help='Name of experiment')
parser.add_argument('--run', dest='run',
                    required=True, type=str,
                    help='Name of run within the experiment')

# Hyperparameters for experiment
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')

parser.add_argument('--dataset-uri', dest='dataset_uri',
                    required=True, type=str,
                    help='Location of the dataset')

parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str,
                    help='Storage location for the model')
args = parser.parse_args()

def get_data(dataset_uri, execution):
    # get the training data
    
    dataset_artifact = aiplatform.Artifact.create(
        schema_title="system.Dataset", display_name="example_dataset", uri=dataset_uri
    )
    
    execution.assign_input_artifacts([dataset_artifact])

    return None

def get_model():
    # get or create the model architecture
    return None

def train_model(dataset, model, epochs):
    aiplatform.log_params({"epochs": epochs})
    # train the model
    return model

def save_model(model, model_dir, execution):
    # save the model
    
    model_artifact = aiplatform.Artifact.create(
        schema_title="system.Model", display_name="example_model", uri=model_dir
    )
    execution.assign_output_artifacts([model_artifact])

# Create a run within the experiment
aiplatform.init(experiment=args.experiment)
aiplatform.start_run(args.run)

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    dataset = get_data(args.dataset_uri, execution)
    model = get_model()
    model = train_model(dataset, model, args.epochs)
    save_model(model, args.model_dir, execution)
    
    # Store the lineage link in the experiment
    aiplatform.log_metrics({"lineage": execution.get_output_artifacts()[0].lineage_console_uri})

aiplatform.end_run()

#### Store training script on your Cloud Storage bucket

Next, you package the training folder into a compressed tar ball, and then store it in your Cloud Storage bucket.

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer.tar.gz

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.

- `python_package_gcs_uri`: The location of the Python training package as a tarball.
- `python_module_name`: The relative path to the training script in the Python package.

*Note:* There is no requirements parameter. You specify any requirements in the `setup.py` script in your Python package.

In [ ]:
DISPLAY_NAME = "example_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
)

#### Run the custom training job

Next, you run the custom training job to start the training job by invoking the method `run()`, with the following parameters:

- `args`: The arguments to pass to the training script
    - `model_dir`: The Cloud Storage location to store the model.
    - `dataset_uri`: The Cloud Storage location of the dataset.
    - `epochs`: The number of epochs (hyperparameter).
    - `experiment`: The name of the experiment.
    - `run`: The name of the run within the experiment.
- `replica_count`: The number of VM instances.
- `machine_type`: The machine type for each VM instance.

In [ ]:
CMDARGS = [
    "--model-dir=" + BUCKET_URI,
    "--dataset-uri=gs://example/foo.csv",
    "--epochs=5",
    f"--experiment={EXPERIMENT_NAME}",
    "--run=run-1",
]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

#### Get the experiment results

Next, you use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In this example, you stored the resource URI to the lineage as a metric value `lineage` in the execution run.

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

#### Visualize the artifact lineage

Next, open the link below to visualize the artifact lineage.

In [ ]:
print("Open the following link", experiment_df["metric.lineage"][0])

#### Delete the custom training job

You can delete your custom training job using the `delete()` method.

In [ ]:
job.delete()

#### Delete the experiment

Since the experiment was created within `Vertex AI Training`, to delete the experiment you use the `list()` method to obtain all the experiments for the project, and then filter on the experiment name.

In [ ]:
experiments = aiplatform.Experiment.list()
for experiment in experiments:
    if experiment.name == EXPERIMENT_NAME:
        experiment.delete()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
! rm -rf custom

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}